In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located

import os
from bs4 import BeautifulSoup
import time
import pandas as pd

In [5]:
def get_info(fund_name):
    path = os.getcwd()
    browser = webdriver.Chrome(executable_path = path+"/chromedriver")
    browser.get("https://www.sec.gov/edgar/search/?r=el#/dateRange=all&startdt=2000-01-01&enddt=2020-10-07&category=all&locationType=located&locationCode=all&entityName=" + fund_name+"&filter_forms=N-Q")
    time.sleep(1)
    end = False
    fund_list = []
    filed_list= []
    enddate_list = []
    entity_list = []
    cik_list = []
    while(end != True):
        cik_checkbox = browser.find_element_by_id("col-cik")
        cik_checkbox.click()
        nextPage = browser.find_element_by_css_selector("a[data-value='nextPage']")
        fundname = browser.find_elements_by_class_name('preview-file')
        filed = browser.find_elements_by_class_name('filed')
        enddate = browser.find_elements_by_class_name('enddate')
        entity = browser.find_elements_by_class_name('entity-name')
        cik = browser.find_elements_by_class_name('cik')
        for i in fundname:
            fund_list.append(i.text)
        for i in filed:
            filed_list.append(i.text)
        for i in enddate:
            enddate_list.append(i.text)
        for i in entity:
            entity_list.append(i.text)
        for i in cik:
            cik_list.append(i.text[4:])
        try:
            nextPage.click()
            time.sleep(0.5)
        except:
            end = True
    browser.close() 
    return fund_list,filed_list,enddate_list,entity_list, cik_list

In [6]:
def write_to_csv(fund_name):
    fund_list,filed_list,enddate_list,entity_list,cik_list = get_info(fund_name)
    uniqueCIK = set(cik_list)
    data_dict = {}
    data_dict["endDate"] = enddate_list[1:]
    data_dict["filedDate"] = filed_list[1:]
    data_dict["entity"] = entity_list[1:]
    data_dict["cik"] = cik_list[1:]
    df = pd.DataFrame(data_dict)
    df.to_csv("data/" + fund_name + ".csv")
    
    cik_dic = {}
    cik_unique_list = list(uniqueCIK)
    cik_dic['CIK'] = cik_unique_list[1:]
    df_cik = pd.DataFrame(cik_dic)
    df_cik.to_csv("CIK/" + fund_name + "_CIK.csv")

Either input the fundname one by one:

In [ ]:
fundname = 'wasatch'
write_to_csv(fundname)

Or put them in a list:

In [16]:
fundlist = ["t rowe price","vanguard","wasatch"]
for fund in fundlist:
    write_to_csv(fund)